# C1 W3 Group 8 - Task 4

In [ ]:
from tqdm import tqdm
from PIL import Image

from src.background import get_painting_masks, crop_image_by_mask
from src.data import GT_QSD2_W3_LIST
from src.denoising import denoise_image
from src.paths import QSD2_W3_PATH, WEEK_3_RESULTS_PATH, BBDD_PATH
from src.retrieval import find_top_k_similar_images
from src.utils import show_images

In [ ]:
query_d2_image_PIL_list = [Image.open(query_img_path) for query_img_path in sorted(QSD2_W3_PATH.glob("*.jpg"))]  # Load once
query_d2_mask_PIL_list = [Image.open(query_img_path) for query_img_path in sorted(QSD2_W3_PATH.glob("*.png"))]  # Load once
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in sorted(BBDD_PATH.glob("*.jpg"))]  # Load once

In [ ]:
# Apply median filter
query_d2_image_PIL_list = [denoise_image(image) for image in query_d2_image_PIL_list]

In [ ]:
# Get painting masks
painting_masks_list = []
pbar = tqdm(zip(sorted(QSD2_W3_PATH.glob("*.jpg")), query_d2_image_PIL_list, query_d2_mask_PIL_list))
for name, image, gt_mask in pbar:
    pbar.set_description(f"Processing {name.stem}")
    painting_masks = get_painting_masks(image)    
    painting_masks_list.append(painting_masks)

In [ ]:
cropped_query_image_list = []
pbar = tqdm(zip(sorted(QSD2_W3_PATH.glob("*.jpg")), query_d2_image_PIL_list, query_d2_mask_PIL_list, painting_masks_list))
for name, image, gt_mask, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    image_list = crop_image_by_mask(image, predicted_mask)
    cropped_query_image_list.append(image_list)

In [ ]:
retrieved_images = find_top_k_similar_images(cropped_query_image_list, database_image_PIL_list)
print(retrieved_images)

In [ ]:
# TODO REMOVE!

import numpy as np
from src.metrics import MeanAveragePrecisionAtK

m = MeanAveragePrecisionAtK()
k = 5

p = 0
n = 0

pbar = tqdm(zip(sorted(QSD2_W3_PATH.glob("*.jpg")), retrieved_images, GT_QSD2_W3_LIST))
for name, pred, gt in pbar:
    pbar.set_description(f"Processing {name.stem}")
    for pred_x, gt_x in zip(pred, gt):
        gt_x = np.array([[gt_x]])
        pred_x = np.array([pred_x])
        res = m.compute(gt_x, pred_x, k=k)
        p += res
        n += 1

print(p/n)